In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install openpyxl




[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import os

# 시간 정보를 날짜로 변환하는 함수
def convert_time_to_date(time):
    now = datetime.now()
    
    if "분 전" in time:
        return now.strftime("%Y-%m-%d")
    elif "시간 전" in time:
        hours_ago = int(time.split("시간 전")[0])
        return (now - timedelta(hours=hours_ago)).strftime("%Y-%m-%d")
    elif "일 전" in time:
        days_ago = int(time.split("일 전")[0])
        return (now - timedelta(days=days_ago)).strftime("%Y-%m-%d")
    elif "주 전" in time:
        weeks_ago = int(time.split("주 전")[0])
        return (now - timedelta(weeks=weeks_ago)).strftime("%Y-%m-%d")
    else:
        return time

# 크롤링 함수
def crawl_news(query):
    options = Options()
    options.add_argument("start-maximized")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    print(f"'{query}' 검색에 대한 브라우저를 열었습니다.")
    
    results = []
    
    try:
        # 네이버 뉴스 페이지로 이동

        url = f"https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2019.01.01&de=2024.08.26&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1023&nso=so%3Ar%2Cp%3Afrom20190101to20240826&is_sug_officeid=0&office_category=0&service_area=0"
        driver.get(url)
        print(f"네이버 뉴스 페이지로 이동했습니다. 검색어: {query}")

        # 첫 번째 기사가 로드될 때까지 대기
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".list_news .bx")))
        print("첫 번째 기사 로드 완료")

        current_articles_count = 0
        page_count = 1

        while True:
            print(f"현재 페이지: {page_count}, 로드된 기사 수: {current_articles_count}")
            articles = driver.find_elements(By.CSS_SELECTOR, ".list_news .bx")
            current_articles_count = len(articles)

            # 스크롤을 내려 더 많은 기사를 로드
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(4)

            articles = driver.find_elements(By.CSS_SELECTOR, ".list_news .bx")

            for i in range(len(results), len(articles)):
                article = articles[i]

                try:
                    print(f"기사 {i+1} 크롤링 중...")
                    title = article.find_element(By.CSS_SELECTOR, ".news_tit").text
                    naver_link = article.find_element(By.CSS_SELECTOR, "div.info_group a[href^='https://n.news.naver.com']").get_attribute("href")

                    time_elements = article.find_elements(By.CSS_SELECTOR, "span.info")
                    time_text = time_elements[1].text if len(time_elements) > 1 else time_elements[0].text if len(time_elements) == 1 else "시간 정보 없음"
                    time_parsed = convert_time_to_date(time_text)

                    print(f"기사 제목: {title}")
                    print(f"기사 링크: {naver_link}")
                    print(f"기사 시간: {time_parsed}")

                    driver.get(naver_link)
                    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "article#dic_area")))
                    content = driver.find_element(By.CSS_SELECTOR, "article#dic_area").text

                    results.append({"title": title, "naverLink": naver_link, "time": time_parsed, "content": content})

                    print("기사 본문 크롤링 완료, 검색 결과 페이지로 돌아갑니다.")
                    driver.back()
                    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".list_news .bx")))

                except Exception as e:
                    print(f"기사 크롤링 중 오류 발생 (index {i}):", e)
                    continue

            if len(articles) == current_articles_count:
                break

            page_count += 1

        print(f"'{query}'에 대한 크롤링 완료. 크롤링한 기사 수: {len(results)}")
        return results

    except Exception as e:
        print("Selenium 작업 중 오류 발생:", e)
        return results  # 현재까지 크롤링된 결과를 반환

    finally:
        # 브라우저를 닫지 않도록 주석 처리
        # driver.quit()
        print(f"'{query}' 검색에 대한 브라우저를 닫지 않습니다. (디버깅용)")

# 검색어 설정
query = '"인사" "정책"'

# 결과 저장할 폴더 경로
folder_name = "조선_인사정책"
os.makedirs(folder_name, exist_ok=True)

# 검색어에 대해 크롤링 수행 및 엑셀로 저장
print(f"===== '{query}' 검색에 대한 크롤링 시작 =====")
news_data = crawl_news(query)
if news_data:
    df = pd.DataFrame(news_data)
    file_path = os.path.join(folder_name, f"{query.replace(' ', '_').replace('\"', '')}_news.xlsx")
    df.to_excel(file_path, index=False)
    print(f"'{query}' 검색 결과가 '{file_path}'에 저장되었습니다.")
else:
    print(f"'{query}' 검색에서 크롤링된 데이터가 없습니다.")
print(f"===== '{query}' 검색에 대한 크롤링 완료 =====\n")


===== '"인사" "정책"' 검색에 대한 크롤링 시작 =====
'"인사" "정책"' 검색에 대한 브라우저를 열었습니다.
네이버 뉴스 페이지로 이동했습니다. 검색어: "인사" "정책"
첫 번째 기사 로드 완료
현재 페이지: 1, 로드된 기사 수: 0
기사 1 크롤링 중...
기사 제목: [인사] 기획재정부 외
기사 링크: https://n.news.naver.com/mnews/article/023/0003854389?sid=102
기사 시간: 2024-08-24
기사 본문 크롤링 완료, 검색 결과 페이지로 돌아갑니다.
기사 2 크롤링 중...
기사 크롤링 중 오류 발생 (index 1): Message: stale element reference: stale element not found
  (Session info: chrome=127.0.6533.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000104981024 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x0000000104979700 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x000000010458882c cxxbridge1$string$len + 88524
3   chromedriver                        0x0000000104596224 cxxbridge1$string$len + 144324
4   chromedriver                        0x0000